# Post Extraction

In [ ]:
import datetime 
import pandas as pd
import zstandard
import os
import json
from datetime import datetime
import logging.handlers
import datetime as dt

In [2]:
files = os.listdir(r'./reddit/subreddits24/')
files = [f for f in files if "submission" in f]
files

['options_submissions.zst',
 'stockstobuytoday_submissions.zst',
 'investing_submissions.zst',
 'wallstreetbets_submissions.zst',
 'stocks_submissions.zst']

In [3]:
wallstreetbets_path = r'./reddit/subreddits24/wallstreetbets_submissions.zst'
stockstobuytoday_path = r'./reddit/subreddits24/stockstobuytoday_submissions.zst'
stocks_path = r'./reddit/subreddits24/stocks_submissions.zst'
investing_path = r'./reddit/subreddits24/investing_submissions.zst'
options_path = r'./reddit/subreddits24/options_submissions.zst'

## Extracting the posts from the raw .zst file

The code is currently demonstrated to work for the r/stocks subreddit but changing it for others is easy and requires only a few parameter changes. This code was manually executed separately as it fills up the memory quite extensively.

In [21]:
# The code is a modified version of this code from GitHub https://github.com/Watchful1/PushshiftDumps/tree/master


log = logging.getLogger("bot")
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler())


def read_and_decode(reader, chunk_size, max_window_size, previous_chunk=None, bytes_read=0):
	chunk = reader.read(chunk_size)
	bytes_read += chunk_size
	if previous_chunk is not None:
		chunk = previous_chunk + chunk
	try:
		return chunk.decode()
	except UnicodeDecodeError:
		if bytes_read > max_window_size:
			raise UnicodeError(f"Unable to decode frame after reading {bytes_read:,} bytes")
		log.info(f"Decoding error with {bytes_read:,} bytes, reading another chunk")
		return read_and_decode(reader, chunk_size, max_window_size, chunk, bytes_read)


def read_lines_zst(file_name):
	with open(file_name, 'rb') as file_handle:
		buffer = ''
		reader = zstandard.ZstdDecompressor(max_window_size=2**31).stream_reader(file_handle)
		while True:
			chunk = read_and_decode(reader, 2**27, (2**29) * 2)

			if not chunk:
				break
			lines = (buffer + chunk).split("\n")

			for line in lines[:-1]:
				yield line, file_handle.tell()

			buffer = lines[-1]

		reader.close()


data_list = []
file_path = stocks_path
file_size = os.stat(file_path).st_size
file_lines = 0
file_bytes_processed = 0
created = None
field = "subreddit"
value = "stocks"
bad_lines = 0
target_date_start = datetime(2017, 1, 1)
target_date_end = datetime(2024, 12, 31)
# try:
for line, file_bytes_processed in read_lines_zst(file_path):
    try:
        #print('1')
        obj = json.loads(line)
        created = datetime.utcfromtimestamp(int(obj['created_utc']))
        #print(created)
        if created < target_date_end and created > target_date_start:
            #print('2')
            if field in obj and obj[field] == value:
                #print('3')
                data_list.append(obj)
        #temp = obj[field] == value
    except (KeyError, json.JSONDecodeError) as err:
        bad_lines += 1
    file_lines += 1
    if file_lines % 100000 == 0:
        log.info(f"{created.strftime('%Y-%m-%d %H:%M:%S')} : {file_lines:,} : {bad_lines:,} : {file_bytes_processed:,}:{(file_bytes_processed / file_size) * 100:.0f}%")

# except Exception as err:
# 	log.info(err)

log.info(f"Complete : {file_lines:,} : {bad_lines:,}")

print('num post extracted:', len(data_list))


/var/folders/qc/7382bn3j7vq46cnb3_4lsqk40000gn/T/ipykernel_29586/2383645643.py:58: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  created = datetime.utcfromtimestamp(int(obj['created_utc']))
2020-03-25 16:43:16 : 100,000 : 0 : 34,997,025:35%
2020-03-25 16:43:16 : 100,000 : 0 : 34,997,025:35%
2020-03-25 16:43:16 : 100,000 : 0 : 34,997,025:35%
2020-03-25 16:43:16 : 100,000 : 0 : 34,997,025:35%
2020-03-25 16:43:16 : 100,000 : 0 : 34,997,025:35%
2020-03-25 16:43:16 : 100,000 : 0 : 34,997,025:35%
2021-02-14 00:43:11 : 200,000 : 0 : 59,770,200:60%
2021-02-14 00:43:11 : 200,000 : 0 : 59,770,200:60%
2021-02-14 00:43:11 : 200,000 : 0 : 59,770,200:60%
2021-02-14 00:43:11 : 200,000 : 0 : 59,770,200:60%
2021-02-14 00:43:11 : 200,000 : 0 : 59,770,200:60%
2021-02-14 00:43:11 : 200,000 : 0 : 59,770,200:60%
2022-08-

num post extracted: 340274


In [22]:
datetime.utcfromtimestamp(int(data_list[0]['created_utc']))

/var/folders/qc/7382bn3j7vq46cnb3_4lsqk40000gn/T/ipykernel_29586/1574632089.py:1: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  datetime.utcfromtimestamp(int(data_list[0]['created_utc']))


datetime.datetime(2017, 1, 1, 3, 6, 21)

In [6]:
data_list[0]

{'all_awardings': [{'award_sub_type': 'GLOBAL',
   'award_type': 'global',
   'awardings_required_to_grant_benefits': None,
   'coin_price': 100,
   'coin_reward': 0,
   'count': 1,
   'days_of_drip_extension': 0,
   'days_of_premium': 0,
   'description': "Shows the Silver Award... and that's it.",
   'end_date': None,
   'giver_coin_reward': None,
   'icon_format': None,
   'icon_height': 512,
   'icon_url': 'https://www.redditstatic.com/gold/awards/icon/silver_512.png',
   'icon_width': 512,
   'id': 'gid_1',
   'is_enabled': True,
   'is_new': False,
   'name': 'Silver',
   'penny_donate': None,
   'penny_price': None,
   'resized_icons': [{'height': 16,
     'url': 'https://www.redditstatic.com/gold/awards/icon/silver_16.png',
     'width': 16},
    {'height': 32,
     'url': 'https://www.redditstatic.com/gold/awards/icon/silver_32.png',
     'width': 32},
    {'height': 48,
     'url': 'https://www.redditstatic.com/gold/awards/icon/silver_48.png',
     'width': 48},
    {'height'

## Building a DataFrame from the extracted data

In [23]:
df = pd.DataFrame()
post_ids = []
dates = []
timestamps = []
texts = []
usernames = []
scores = []
comment_nums = []
titles = []

for post in data_list:
   post_id = post['id']
   text = post['selftext']
   title = post['title']
   score = post['score']
   comment_num = post['num_comments']
   username = post['author']
   time = datetime.fromtimestamp(int(post['created_utc']), tz=dt.UTC)
   date = time.date()
   timestamp = time.time()
   
   post_ids.append(post_id)
   dates.append(date)
   timestamps.append(timestamp)
   texts.append(text)
   usernames.append(username)
   scores.append(score)
   comment_nums.append(comment_num)
   titles.append(title)
   
df['post_id'] = post_ids
df['username'] = usernames
df['date'] = dates
df['timestamp'] = timestamps
df['title'] = titles
df['text'] = texts
df['score'] = scores
df['comment_num'] = comment_nums

df['date'] = pd.to_datetime(df['date'])
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%H:%M:%S').dt.time

df.head(50) 

,post_id,username,date,timestamp,title,text,score,comment_num
0,5ld8mn,funktaxi,2017-01-01,03:06:21,Selling my AAPL position because of Itunes/App...,It's New Year's Eve. I'm not drunk. I want t...,0,12
1,5ldaqw,frozenflame93,2017-01-01,03:22:48,Help,"Hello Reddit, \n\nI will try to keep this shor...",2,10
2,5ldlkf,[deleted],2017-01-01,04:48:06,The Trump Effect,[deleted],12,7
3,5ldmgb,downdownwhatup,2017-01-01,04:55:09,options for starting an investment vehicle?,I have been trading for about 3 years now and ...,5,11
4,5ldqsy,[deleted],2017-01-01,05:26:55,need some advice for a rookie that wants to ge...,[deleted],12,11
5,5ldw08,scottnow,2017-01-01,06:08:34,Exxon Mobil (XOM) vs. Chevron (CVX) in 2017,Curious as to the thought of /r/stocks on Chev...,22,5
6,5le77j,Share_Tipper,2017-01-01,07:47:57,Is this all US people?,[removed],1,1
7,5legdy,mitchellosu,2017-01-01,09:15:25,How do I invest in the failure of brick and mo...,The first quarter is usually when store closin...,2,11
8,5lekzd,InvestorsHangout,2017-01-01,10:04:33,Investors Hangout Stock Analysis Full Live Str...,https://www.youtube.com/watch?v=KsJcQhO1rso\n\...,8,0
9,5lexlv,learner1314,2017-01-01,12:17:04,"If a stock is $2.38 at close, and I issue a bu...",So say last week Friday a stock closed at $2.3...,7,5


In [ ]:
if os.path.exists('./submissions'):
    df.to_csv('stocks_submissions.csv', index=False) 
else:
    os.makedirs('./submissions')
    df.to_csv('./submissions/stocks_submissions.csv', index=False)

: 